<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebooks/PAWS_DR_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Drive
from google.colab import drive
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

In [2]:
#drive.mount('/content/drive')
#auth.authenticate_user()

In [3]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install --quiet -v --no-cache-dir ./

Cloning into 'apex'...
remote: Enumerating objects: 8054, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 8054 (delta 68), reused 97 (delta 44), pack-reused 7913
Receiving objects: 100% (8054/8054), 14.11 MiB | 22.61 MiB/s, done.
Resolving deltas: 100% (5467/5467), done.
/content/apex
Processing /content/apex
  Created wheel for apex: filename=apex-0.1-cp37-none-any.whl size=204709 sha256=bd1f0766b9f00b956681182b4b8b330e3dd9444c32977ddc0839319a7095e919
  Stored in directory: /tmp/pip-ephem-wheel-cache-c9ahq1a9/wheels/b1/3a/aa/d84906eaab780ae580c7a5686a33bf2820d8590ac3b60d5967
Successfully built apex


In [4]:
pip install PyYAML

In [5]:
cd ..

/content


In [6]:
!git clone -b feature/DR-images-v2 https://github.com/jmarrietar/suncet.git

Cloning into 'suncet'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 290 (delta 169), reused 217 (delta 102), pack-reused 0
Receiving objects: 100% (290/290), 1.10 MiB | 8.29 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [7]:
cd suncet

/content/suncet


In [10]:
pwd

'/content/suncet'

In [9]:
!mkdir datasets

In [11]:
pip install -U PyYAML

     |████████████████████████████████| 645kB 15.7MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [12]:
!mkdir logs

In [13]:
!mkdir datasets/dr

In [14]:
!python download.py -d sample@1000

Downloading...
From: https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2
To: /content/suncet/datasets/dr/sample@1000.zip
108MB [00:01, 82.6MB/s]


In [21]:
!python download.py -d sample@20000

Downloading...
From: https://drive.google.com/uc?id=1MTDpLzpmhSiZq2jSdmHx2UDPn9FC8gzO
To: /content/suncet/datasets/dr/sample@20000.zip
1.08GB [00:12, 89.3MB/s]


In [15]:
import argparse

import torch.multiprocessing as mp

import pprint
import yaml

from src.paws_train import main as paws
from src.suncet_train import main as suncet
from src.fine_tune import main as fine_tune
from src.snn_fine_tune import main as snn_fine_tune

from src.utils import init_distributed

In [16]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--fname', type=str,
    help='name of config file to load',
    default='configs.yaml')
parser.add_argument(
    '--devices', type=str, nargs='+', default=['cuda:0'],
    help='which devices to use on local machine')
parser.add_argument(
    '--sel', type=str,
    help='which script to run',
    choices=[
        'paws_train',
        'suncet_train',
        'fine_tune',
        'snn_fine_tune'
    ])

_StoreAction(option_strings=['--sel'], dest='sel', nargs=None, const=None, default=None, type=<class 'str'>, choices=['paws_train', 'suncet_train', 'fine_tune', 'snn_fine_tune'], help='which script to run', metavar=None)

In [ ]:
def process_main(rank, sel, fname, world_size, devices):
    import os
    #os.environ['CUDA_VISIBLE_DEVICES'] = str(devices[rank].split(':')[-1])
    os.environ['CUDA_VISIBLE_DEVICES'] = str(args.devices[rank].split(':')[-1]) # YO

    import logging
    logging.basicConfig()
    logger = logging.getLogger()

    logger.info(f'called-params {sel} {fname}')

    # -- load script params
    params = None
    with open(fname, 'r') as y_file:
        params = yaml.load(y_file, Loader=yaml.FullLoader)
        logger.info('loaded params...')
        if rank == 0:
            pp = pprint.PrettyPrinter(indent=4)
            pp.pprint(params)

    print("loggin folder {}".format(params['logging']['folder']))

    if rank == 0:
        dump = os.path.join(params['logging']['folder'], f'params-{sel}.yaml')
        with open(dump, 'w') as f:
            yaml.dump(params, f)

    world_size, rank = init_distributed(rank_and_world_size=(rank, world_size))

    # -- make sure all processes correctly initialized torch-distributed
    logger.info(f'Running {sel} (rank: {rank}/{world_size})')

    # -- turn off info-logging for ranks > 0, otherwise too much std output
    if rank == 0:
        logger.setLevel(logging.INFO)
    else:
        logger.setLevel(logging.ERROR)

    if sel == 'paws_train':
        return paws(params)
    elif sel == 'suncet_train':
        return suncet(params)
    elif sel == 'fine_tune':
        return fine_tune(params)
    elif sel == 'snn_fine_tune':
        return snn_fine_tune(params)

In [ ]:
args = parser.parse_args(['--sel', 'paws_train',
                            '--fname', 'configs/paws/dr_train.yaml'
])

In [ ]:
num_gpus = len(args.devices)

In [ ]:
process_main(0, args.sel, args.fname, num_gpus, args.devices)

INFO:root:called-params paws_train configs/paws/dr_train.yaml
INFO:root:loaded params...
{   'criterion': {   'classes_per_batch': 2,
                     'me_max': True,
                     'sharpen': 0.25,
                     'supervised_imgs_per_class': 32,
                     'supervised_views': 1,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 32},
    'data': {   'color_jitter_strength': 1.0,
                'data_seed': None,
                'dataset': 'dr',
                'image_folder': 'dr/sample@1000/',
                'label_smoothing': 0.1,
                'multicrop': 6,
                'normalize': True,
                'root_path': 'datasets/',
                'subset_path': 'dr_subsets',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.9},
    'logging': {'folder': 'logs/', 'write_tag': 'paws'},
    'meta': {   'copy_data': True,
                'device': 'cuda:0',
                'load_

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


INFO:root:len.supervised_loader: 31


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


INFO:root:[1,     0] loss: 0.001 (0.694 -0.693) (5087 ms; 9980 ms)


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


INFO:root:[1,    10] loss: 0.002 (0.693 -0.691) (1040 ms; 917 ms)
INFO:root:[1,    10] lr_stats: 0.215 (7.43e-05, 5.87e+00)
INFO:root:[1,    20] loss: 0.003 (0.694 -0.691) (955 ms; 487 ms)
INFO:root:[1,    20] lr_stats: 0.379 (1.56e-04, 1.02e+01)
INFO:root:[1,    30] loss: 0.004 (0.695 -0.691) (766 ms; 332 ms)
INFO:root:[1,    30] lr_stats: 0.557 (3.17e-04, 1.47e+01)
INFO:root:avg. loss 0.004
INFO:root:updating "best" checkpoint
INFO:root:Epoch 2
INFO:root:len.supervised_loader: 31
INFO:root:[2,     0] loss: 0.007 (0.696 -0.689) (1165 ms; 10711 ms)
INFO:root:[2,     0] lr_stats: 0.155 (8.87e-05, 4.11e+00)
INFO:root:[2,    10] loss: 0.008 (0.696 -0.688) (864 ms; 1014 ms)
INFO:root:[2,    10] lr_stats: 0.176 (1.57e-04, 4.49e+00)
INFO:root:[2,    20] loss: 0.005 (0.692 -0.687) (912 ms; 556 ms)
INFO:root:[2,    20] lr_stats: 0.398 (7.13e-04, 9.37e+00)
INFO:root:[2,    30] loss: 0.004 (0.693 -0.688) (766 ms; 379 ms)
INFO:root:[2,    30] lr_stats: 1.233 (2.98e-03, 2.48e+01)
INFO:root:avg. lo

KeyboardInterrupt: ignored

## Scripts

In [ ]:
# Pre-train 
!python main.py --sel paws_train --fname configs/paws/dr_train.yaml

INFO:root:called-params paws_train configs/paws/dr_train.yaml
INFO:root:loaded params...
{   'criterion': {   'classes_per_batch': 2,
                     'me_max': True,
                     'sharpen': 0.25,
                     'supervised_imgs_per_class': 32,
                     'supervised_views': 1,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 32},
    'data': {   'color_jitter_strength': 1.0,
                'data_seed': None,
                'dataset': 'dr',
                'label_smoothing': 0.1,
                'multicrop': 6,
                'normalize': True,
                'root_path': 'datasets/',
                's_image_folder': 'dr/sample@1000/',
                'subset_path': 'dr_subsets',
                'u_image_folder': 'dr/sample@20000/',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.9},
    'logging': {'folder': 'logs/', 'write_tag': 'paws'},
    'meta': {   'copy_data': True,
  